## Climate Stress Testing & Portfolio Simulation with ML

In [234]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="n_jobs value 1 overridden to 1 by setting random_state"
)

In [235]:
# %%
from pathlib import Path
import warnings

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

import umap
import hdbscan

# Global configuration
DEFAULT_SEED = 42
N_MC_PATHS = 500
DECARB_TARGET_RANGE = (0.01, 0.06)
SCENARIO_NAMES = ("orderly", "disorderly", "hothouse")
DATA_DIR = Path.cwd()
OUTPUT_DIR = DATA_DIR / "outputs"
DATA_FILES = {
    "low": "low_risk_opportunities.csv",
    "critical": "critical_risk_assets.csv",
    "company": "company_carbon_exposure.csv",
    "divest": "divestment_candidates.csv",
}

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 160)


def load_dataset(key: str, data_dir: Path = DATA_DIR) -> pd.DataFrame:
    """Return the requested dataset from disk with a helpful error if missing."""
    try:
        filename = DATA_FILES[key]
    except KeyError as exc:
        raise KeyError(
            f"Unknown dataset key '{key}'. Available keys: {list(DATA_FILES)}"
        ) from exc

    path = data_dir / filename
    if not path.exists():
        raise FileNotFoundError(f"Expected dataset at {path}, but it was not found.")
    return pd.read_csv(path)


def summarize_dataframe(df: pd.DataFrame, *, name: str) -> pd.Series:
    """Compact summary with shape, column types and average missingness."""
    numeric_cols = df.select_dtypes(include=[np.number]).shape[1]
    categorical_cols = df.select_dtypes(exclude=[np.number]).shape[1]
    missing_pct = float(df.isna().mean().mean()) * 100
    return pd.Series(
        {
            "rows": df.shape[0],
            "columns": df.shape[1],
            "numeric_cols": numeric_cols,
            "categorical_cols": categorical_cols,
            "avg_missing_pct": round(missing_pct, 2),
        },
        name=name,
    )


def preview_dataset(df: pd.DataFrame, *, name: str, n: int = 5) -> None:
    """Display a quick preview for interactive exploration."""
    display(
        df.head(n).style.set_caption(f"{name} – first {n} rows"),
    )


def export_dataframe(df: pd.DataFrame, filename: str, *, index: bool = False) -> Path:
    """Persist dataframe to outputs directory, preferring Parquet with CSV fallback."""
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    parquet_path = OUTPUT_DIR / f"{filename}.parquet"
    try:
        df.to_parquet(parquet_path, index=index)
        return parquet_path
    except Exception as exc:  # noqa: BLE001
        warnings.warn(
            f"Parquet export failed ({exc}); falling back to CSV.",
            RuntimeWarning,
        )
        csv_path = OUTPUT_DIR / f"{filename}.csv"
        df.to_csv(csv_path, index=index)
        return csv_path


# Data
low = load_dataset("low")  # [file:1]
crit = load_dataset("critical")  # [file:2]
comp = load_dataset("company")  # [file:4]
div = load_dataset("divest")  # [file:3]

comp.columns = [c.strip() for c in comp.columns]

### 1. Quick data overview

In [236]:
# %%
summaries = pd.concat(
    [
        summarize_dataframe(low, name="Low-risk mines"),
        summarize_dataframe(crit, name="Critical-risk mines"),
        summarize_dataframe(comp, name="Company exposures"),
        summarize_dataframe(div, name="Divest candidates"),
    ]
, axis=1).T

display(summaries.style.set_caption("Dataset overview"))

for name, df in (
    ("Low-risk mines", low),
    ("Critical-risk mines", crit),
    ("Company exposures", comp),
    ("Divest candidates", div),
):
    preview_dataset(df, name=name, n=3)


,rows,columns,numeric_cols,categorical_cols,avg_missing_pct
Low-risk mines,28.000000,8.000000,4.000000,4.000000,8.040000
Critical-risk mines,21.000000,10.000000,5.000000,5.000000,1.900000
Company exposures,25.000000,10.000000,8.000000,2.000000,0.000000
Divest candidates,20.000000,4.000000,3.000000,1.000000,0.000000


,Mine,Country,Parent,Intensity,Capacity,Production,Cost@$100,Confidence
0,Abyz Mine,KAZ,Kazakhmys Holding LLP,0.005600,0.680070,132094466.647848,73972901.322795,very low
1,Akchi-Spassky Mine,KAZ,nan,0.005600,0.680070,132094466.647848,73972901.322795,very low
2,Aralchinsky Mine,KAZ,nan,0.005600,0.680070,132094466.647848,73972901.322795,very low


,Mine,Country,Parent,Type,Emissions,Intensity,Cost@$100,Cost@$200,Capacity,Confidence
0,Mount Isa Operation,AUS,Government of Qatar,Both,204750.000000,0.045500,20475000.000000,40950000.000000,0.692308,high
1,El Salvador Mine,CHL,Codelco Corp,Both,216433.000000,0.300500,21643300.000000,43286600.000000,0.001062,medium
2,Lomas Bayas Mine,CHL,Glencore PLC,Open Pit,1038655.000000,0.045500,103865500.000000,207731000.000000,0.080379,medium


,Company,HQ Country,Mines,Total Emissions (tCO₂),Production (t),$50/t,$100/t,$150/t,$200/t,Portfolio Intensity
0,FreePort-McMoran Inc,USA,11,4858750.019972,468335534.553201,242937500.998614,485875001.997227,728812502.995841,971750003.994455,0.010375
1,Government of Iran,IRN,3,4844845.000000,201868552.000000,242242250.000000,484484500.000000,726726750.000000,968969000.000000,0.024000
2,Qatar Investment Authority,QAT,11,2318213.001601,52153138.035190,115910650.080058,231821300.160116,347731950.240174,463642600.320233,0.044450


,Company,Assets at Risk,Exposure@$100/t,Emissions
0,Government of Iran,3,484484500.000000,4844845.000000
1,FreePort-McMoran Inc,6,438491501.993730,4384915.019937
2,Qatar Investment Authority,5,211210300.074502,2112103.000745


### 2. Scenarios & ML-based decarbonization

#### 2.1 Deterministic NGFS-style scenarios

In [237]:
# %%
YEARS = list(range(2025, 2041))


def make_scenarios(
    years,
    *,
    orderly_start: float = 50,
    orderly_step: float = 10,
    disorderly_floor: float = 40,
    disorderly_pre_break: int = 8,
    disorderly_ramp: float = 20,
    hothouse_step: float = 2,
    decay_orderly: float = 0.04,
    decay_disorderly: float = 0.02,
    decay_hothouse: float = 0.01,
):
    """Create stylized NGFS-like scenarios with explicit controls."""
    t = np.arange(len(years))

    price_orderly = orderly_start + orderly_step * t

    price_disorderly = np.where(
        t < disorderly_pre_break,
        disorderly_floor + hothouse_step * t,
        disorderly_floor + hothouse_step * disorderly_pre_break
        + disorderly_ramp * (t - disorderly_pre_break),
    )

    price_hothouse = disorderly_floor + hothouse_step * t

    factor_orderly = (1 - decay_orderly) ** t
    factor_disorderly = (1 - decay_disorderly) ** t
    factor_hothouse = (1 - decay_hothouse) ** t

    idx = pd.Index(years, name="Year")
    return {
        "orderly": pd.DataFrame(
            {
                "Price": price_orderly,
                "Intensity_factor": factor_orderly,
            },
            index=idx,
        ),
        "disorderly": pd.DataFrame(
            {
                "Price": price_disorderly,
                "Intensity_factor": factor_disorderly,
            },
            index=idx,
        ),
        "hothouse": pd.DataFrame(
            {
                "Price": price_hothouse,
                "Intensity_factor": factor_hothouse,
            },
            index=idx,
        ),
    }


scenarios = make_scenarios(YEARS)

In [238]:
# %%
scen_frames = []
for name, df_s in scenarios.items():
    tmp = df_s.reset_index()
    tmp["Scenario"] = name
    scen_frames.append(tmp)
scen_plot = pd.concat(scen_frames, ignore_index=True)

long_metrics = scen_plot.melt(
    id_vars=["Year", "Scenario"],
    value_vars=["Price", "Intensity_factor"],
    var_name="Metric",
    value_name="Value",
)
long_metrics["Metric"] = long_metrics["Metric"].map(
    {
        "Price": "Carbon price (USD/t)",
        "Intensity_factor": "Global intensity factor",
    }
)

fig = px.line(
    long_metrics,
    x="Year",
    y="Value",
    color="Scenario",
    facet_row="Metric",
    height=600,
    title="Scenario trajectories: price vs global intensity",
)
fig.update_yaxes(matches=None)
fig.update_traces(mode="lines+markers")
fig.show()

#### 2.2 ML proxy for company-specific decarbonization rates

In [239]:
# %%
comp["emissions_intensity"] = comp["Total Emissions (tCO₂)"] / comp["Production (t)"]  # [file:4]

intensity_min, intensity_max = comp["Portfolio Intensity"].agg(["min", "max"])

nrm_intensity = (comp["Portfolio Intensity"] - intensity_min) / (
    intensity_max - intensity_min
)

low_target, high_target = DECARB_TARGET_RANGE

comp["target_decarb_rate"] = low_target + (high_target - low_target) * nrm_intensity

reg_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("model", LinearRegression()),
    ]
)

X_int = comp[["Portfolio Intensity", "emissions_intensity"]]
y_target = comp["target_decarb_rate"]

reg_pipeline.fit(X_int, y_target)

comp["decarb_rate_ml"] = reg_pipeline.predict(X_int).clip(*DECARB_TARGET_RANGE)
display(comp[["Company", "Portfolio Intensity", "decarb_rate_ml"]].head())

,Company,Portfolio Intensity,decarb_rate_ml
0,FreePort-McMoran Inc,0.010375,0.016875
1,Government of Iran,0.024000,0.028885
2,Qatar Investment Authority,0.044450,0.046911
3,Kazakhmys Holding LLP,0.005600,0.012666
4,The Vanguard Group Inc,0.004528,0.011721


#### 2.3 Company-level trajectories per scenario

In [240]:
# %%
def simulate_company_scenario(row, scen_name, scen_df, years):
    base_e = row["Total Emissions (tCO₂)"]
    r = row["decarb_rate_ml"]
    out = []
    for i, y in enumerate(years):
        p = scen_df.loc[y, "Price"]
        f_scen = scen_df.loc[y, "Intensity_factor"]
        f_co = (1 - r) ** i
        e_t = base_e * f_scen * f_co
        c_t = e_t * p
        out.append({
            "Company": row["Company"],
            "Year": y,
            "Scenario": scen_name,
            "Price": p,
            "Emissions_t": e_t,
            "Cost_t": c_t
        })
    return out

paths_all = []
for scen_name, scen_df in scenarios.items():
    for _, r in comp.iterrows():
        paths_all.extend(simulate_company_scenario(r, scen_name, scen_df, YEARS))

paths_df = pd.DataFrame(paths_all)
paths_df.head()

,Company,Year,Scenario,Price,Emissions_t,Cost_t
0,FreePort-McMoran Inc,2025,orderly,50,4.858750e+06,2.429375e+08
1,FreePort-McMoran Inc,2026,orderly,60,4.585688e+06,2.751413e+08
2,FreePort-McMoran Inc,2027,orderly,70,4.327973e+06,3.029581e+08
3,FreePort-McMoran Inc,2028,orderly,80,4.084741e+06,3.267793e+08
4,FreePort-McMoran Inc,2029,orderly,90,3.855179e+06,3.469661e+08


In [241]:
# %%
top_companies = (
    comp.sort_values("Total Emissions (tCO₂)", ascending=False)
        .head(5)["Company"].tolist()
)

fig = px.line(
    paths_df[paths_df["Company"].isin(top_companies)],
    x="Year", y="Cost_t",
    color="Company",
    facet_col="Scenario",
    facet_col_wrap=1,
    title="Cost trajectories (top 5 emitters) across scenarios"
)
fig.update_traces(mode="lines+markers")
fig.show()

### 3. Portfolio definition & trajectories

#### 3.1 Define portfolio weights

In [242]:
# Example 1: equal weights
comp["w_equal"] = 1.0 / len(comp)

# Example 2: weights proportional to production
comp["w_prod"] = comp["Production (t)"] / comp["Production (t)"].sum()

# Example 3: carbon-aware tilt (inverse portfolio intensity)
inv_intensity = 1 / comp["Portfolio Intensity"].replace(0, np.nan)
inv_intensity = inv_intensity.fillna(inv_intensity.median())
comp["w_carbon_tilt"] = inv_intensity / inv_intensity.sum()

MC_WEIGHT_STRATEGIES = {
    "Equal": "w_equal",
    "Production": "w_prod",
    "Carbon Tilt": "w_carbon_tilt",
}

comp[["Company", "w_equal", "w_prod", "w_carbon_tilt"]].head()

,Company,w_equal,w_prod,w_carbon_tilt
0,FreePort-McMoran Inc,0.04,0.162674,0.032709
1,Government of Iran,0.04,0.070118,0.014139
2,Qatar Investment Authority,0.04,0.018115,0.007634
3,Kazakhmys Holding LLP,0.04,0.140316,0.060597
4,The Vanguard Group Inc,0.04,0.160419,0.074948


#### 3.2 Portfolio trajectories

In [243]:
# %%
def compute_portfolio_trajectory(paths_df, weights_col, comp_df):
    """Aggregate company-level trajectories into a weighted portfolio path."""
    weights = (
        comp_df
        .set_index("Company")[weights_col]
        .rename("weight")
    )

    merged = paths_df.merge(weights, on="Company", how="left")
    if "weight" not in merged.columns:
        missing = set(paths_df["Company"]).difference(weights.index)
        raise ValueError(
            "Column 'weight' missing after merge. "
            f"Missing companies in weights: {sorted(missing)}"
        )

    merged["Cost_weighted"] = merged["Cost_t"] * merged["weight"]
    merged["Emissions_weighted"] = merged["Emissions_t"] * merged["weight"]

    portfolio = (
        merged
        .groupby(["Scenario", "Year"], as_index=False)[
            ["Cost_weighted", "Emissions_weighted"]
        ]
        .sum()
        .rename(
            columns={
                "Cost_weighted": "Portfolio_Cost",
                "Emissions_weighted": "Portfolio_Emissions",
            }
        )
    )
    return portfolio


portfolio_equal = compute_portfolio_trajectory(paths_df, "w_equal", comp)
portfolio_prod = compute_portfolio_trajectory(paths_df, "w_prod", comp)

display(portfolio_equal.head())

,Scenario,Year,Portfolio_Cost,Portfolio_Emissions
0,disorderly,2025,4.756056e+07,1.189014e+06
1,disorderly,2026,4.778763e+07,1.137801e+06
2,disorderly,2027,4.791528e+07,1.088984e+06
3,disorderly,2028,4.795225e+07,1.042440e+06
4,disorderly,2029,4.790664e+07,9.980550e+05


In [244]:
strategy_frames = []
for strategy_name, weight_col in MC_WEIGHT_STRATEGIES.items():
    port = compute_portfolio_trajectory(paths_df, weight_col, comp)
    port["Strategy"] = strategy_name
    strategy_frames.append(port)

port_all = pd.concat(strategy_frames, ignore_index=True)

fig = px.line(
    port_all,
    x="Year",
    y="Portfolio_Cost",
    color="Strategy",
    facet_col="Scenario",
    facet_col_wrap=1,
    title="Portfolio cost trajectories by strategy & scenario"
)
fig.update_traces(mode="lines+markers")
fig.show()

fig = px.line(
    port_all,
    x="Year",
    y="Portfolio_Emissions",
    color="Strategy",
    facet_col="Scenario",
    facet_col_wrap=1,
    title="Portfolio emissions trajectories by strategy & scenario"
)
fig.update_traces(mode="lines+markers")
fig.show()

port_summary = (
    port_all.loc[port_all["Year"].isin([2030, 2040])]
    .pivot_table(
        index=["Scenario", "Strategy"],
        columns="Year",
        values=["Portfolio_Cost", "Portfolio_Emissions"],
    )
)
port_summary.columns = [f"{metric}_{year}" for metric, year in port_summary.columns]
port_summary = port_summary.reset_index()

format_cols = port_summary.select_dtypes(include=[np.number]).columns
style = port_summary.style.format({col: "{0:,.0f}" for col in format_cols})
style = style.set_caption("Key portfolio metrics by strategy")
display(style)

,Scenario,Strategy,Portfolio_Cost_2030,Portfolio_Cost_2040,Portfolio_Emissions_2030,Portfolio_Emissions_2040
0,disorderly,Carbon Tilt,"43,041,560","117,529,341","860,831","599,639"
1,disorderly,Equal,"47,785,922","122,476,123","955,718","624,878"
2,disorderly,Production,"91,839,886","246,473,744","1,836,798","1,257,519"
3,hothouse,Carbon Tilt,"45,282,834","48,879,309","905,657","698,276"
4,hothouse,Equal,"50,274,246","50,936,627","1,005,485","727,666"
5,hothouse,Production,"96,622,202","102,506,030","1,932,444","1,464,372"
6,orderly,Carbon Tilt,"77,650,416","88,023,288","776,504","440,116"
7,orderly,Equal,"86,209,624","91,728,167","862,096","458,641"
8,orderly,Production,"165,686,498","184,595,855","1,656,865","922,979"


### 4. Monte Carlo climate scenarios & Climate VaR

#### 4.1 Randomized price & decarb paths

In [245]:
# %%
def sample_price_intensity_paths(
    years,
    base_scen_df,
    *,
    rng,
    price_vol: float = 0.25,
    intensity_vol: float = 0.15,
    corr: float = 0.3,
    intensity_bounds: tuple[float, float] = (0.6, 1.4),
    draws: np.ndarray | None = None,
):
    """Generate correlated carbon price and intensity shock paths."""
    mean = np.zeros(2)
    cov = np.array(
        [
            [price_vol ** 2, corr * price_vol * intensity_vol],
            [corr * price_vol * intensity_vol, intensity_vol ** 2],
        ]
    )

    if draws is None:
        draws = rng.multivariate_normal(mean, cov, size=len(years))
    else:
        draws = np.asarray(draws)

    price_base = base_scen_df["Price"].values
    intensity_base = base_scen_df["Intensity_factor"].values

    price_shock = np.exp(draws[:, 0] - 0.5 * price_vol ** 2)
    price_path = price_base * price_shock

    intensity_noise = np.clip(1 + draws[:, 1], *intensity_bounds)
    intensity_path = intensity_base * intensity_noise

    return (
        pd.Series(price_path, index=years, name="Price"),
        intensity_path,
        draws,
    )


#### 4.2 Monte Carlo simulation at portfolio level

In [246]:
# %%
def simulate_portfolio_mc(
    comp_df,
    base_scen_df,
    years,
    weights_col,
    *,
    n_paths: int = N_MC_PATHS,
    seed: int | None = DEFAULT_SEED,
    price_vol: float = 0.25,
    intensity_vol: float = 0.15,
    price_intensity_corr: float = 0.3,
    intensity_bounds: tuple[float, float] = (0.6, 1.4),
    antithetic: bool = True,
    control_variate: bool = True,
    strategy_name: str = "Equal",
):
    """Simulate Monte Carlo carbon price and intensity paths for the portfolio."""
    rng = np.random.default_rng(seed)
    results = []

    grouped = (
        comp_df
        .groupby("Company")
        .agg(
            weight=(weights_col, "sum"),
            emissions=("Total Emissions (tCO₂)", "sum"),
            decarb=("decarb_rate_ml", "mean"),
        )
    )

    if grouped.empty:
        raise ValueError("Input dataframe produced no company aggregates for simulation.")

    grouped["weight"] = grouped["weight"].fillna(0)
    weight_total = grouped["weight"].sum()
    if weight_total <= 0:
        raise ValueError("Sum of weights is zero; cannot simulate portfolio.")
    grouped["weight"] = grouped["weight"] / weight_total

    grouped = grouped.dropna(subset=["emissions", "decarb"])
    if grouped.empty:
        raise ValueError("No companies with emissions and decarb data after cleaning.")

    weights_arr = grouped["weight"].values
    base_emissions_arr = grouped["emissions"].values
    decarb_arr = grouped["decarb"].values

    base_prices = base_scen_df.loc[years, "Price"].values
    base_factor = base_scen_df.loc[years, "Intensity_factor"].values

    price_sensitivity = np.zeros(len(years))
    for step in range(len(years)):
        decay = (1 - decarb_arr) ** step
        emissions_year = base_emissions_arr * base_factor[step] * decay
        price_sensitivity[step] = np.sum(emissions_year * weights_arr)

    cov = np.array([
        [price_vol ** 2, price_intensity_corr * price_vol * intensity_vol],
        [price_intensity_corr * price_vol * intensity_vol, intensity_vol ** 2],
    ])

    total_paths = n_paths if not antithetic else int(np.ceil(n_paths / 2))

    for pair_id in range(total_paths):
        draws = rng.multivariate_normal(np.zeros(2), cov, size=len(years))

        scenarios_draws = [draws]
        if antithetic:
            scenarios_draws.append(-draws)

        for local_id, current_draws in enumerate(scenarios_draws):
            path_id = pair_id * (2 if antithetic else 1) + local_id
            if path_id >= n_paths:
                break

            price_path, intensity_path, _ = sample_price_intensity_paths(
                years,
                base_scen_df,
                rng=rng,
                price_vol=price_vol,
                intensity_vol=intensity_vol,
                corr=price_intensity_corr,
                intensity_bounds=intensity_bounds,
                draws=current_draws,
            )

            for step, year in enumerate(years):
                price_year = price_path.iloc[step]
                factor_year = intensity_path[step]

                decay = (1 - decarb_arr) ** step
                emissions_year = base_emissions_arr * factor_year * decay
                cost_year = np.sum(emissions_year * weights_arr) * price_year

                record = {
                    "MC_id": path_id,
                    "Year": year,
                    "Scenario": base_scen_df.name,
                    "Strategy": strategy_name,
                    "Portfolio_Cost": cost_year,
                    "Price": price_year,
                    "Intensity_factor": factor_year,
                }

                if control_variate:
                    record["Portfolio_Cost_CV"] = (
                        cost_year
                        - price_sensitivity[step] * (price_year - base_prices[step])
                    )

                results.append(record)
    return pd.DataFrame(results)


mc_experiments = []
for strategy_name, weight_col in MC_WEIGHT_STRATEGIES.items():
    for scen_name, scen_df in scenarios.items():
        scen_df = scen_df.copy()
        scen_df.name = scen_name
        mc_sim = simulate_portfolio_mc(
            comp,
            scen_df,
            YEARS,
            weight_col,
            price_vol=0.2,
            intensity_vol=0.1,
            price_intensity_corr=0.25,
            n_paths=N_MC_PATHS,
            strategy_name=strategy_name,
        )
        mc_experiments.append(mc_sim)

mc_results = pd.concat(mc_experiments, ignore_index=True)
mc_results.head()

,MC_id,Year,Scenario,Strategy,Portfolio_Cost,Price,Intensity_factor,Portfolio_Cost_CV
0,0,2025,orderly,Equal,4.968823e+07,46.862980,0.891737,5.341819e+07
1,0,2026,orderly,Equal,5.923005e+07,49.911692,1.022123,7.047419e+07
2,0,2027,orderly,Equal,1.014512e+08,103.290468,0.866233,6.666363e+07
3,0,2028,orderly,Equal,7.270559e+07,76.814222,0.854602,7.582730e+07
4,0,2029,orderly,Equal,7.581021e+07,89.680931,0.781254,7.610344e+07


#### 4.3 Climate VaR at horizon

In [247]:
# %%
horizon = 2040
alpha = 0.95

cost_column = (
    "Portfolio_Cost_CV"
    if "Portfolio_Cost_CV" in mc_results.columns
    else "Portfolio_Cost"
)

var_summary = []
for (scen_name, strategy), group in mc_results.groupby(["Scenario", "Strategy"]):
    horizon_slice = group[group["Year"] == horizon]
    horizon_costs = horizon_slice[cost_column].values
    raw_costs = horizon_slice["Portfolio_Cost"].values

    if len(horizon_costs) == 0:
        continue

    VaR = np.quantile(horizon_costs, alpha)
    CVaR = horizon_costs[horizon_costs >= VaR].mean()

    var_summary.append({
        "Scenario": scen_name,
        "Strategy": strategy,
        "Cost_metric": cost_column,
        "VaR_95": VaR,
        "CVaR_95": CVaR,
        "Mean": horizon_costs.mean(),
        "Std": horizon_costs.std(),
        "Raw_Mean": raw_costs.mean(),
        "Raw_Std": raw_costs.std(),
    })

var_df = (
    pd.DataFrame(var_summary)
    .sort_values(["Scenario", "Strategy"])
    .reset_index(drop=True)
)

format_cols = var_df.select_dtypes(include=[np.number]).columns
style = var_df.style.format({col: "{0:,.0f}" for col in format_cols})
style = style.set_caption(
    f"Portfolio risk metrics at Year {horizon} (alpha={alpha:.2f}) using {cost_column}"
)
display(style)

,Scenario,Strategy,Cost_metric,VaR_95,CVaR_95,Mean,Std,Raw_Mean,Raw_Std
0,disorderly,Carbon Tilt,Portfolio_Cost_CV,"140,762,839","146,892,239","118,266,150","12,258,841","118,517,355","31,263,316"
1,disorderly,Equal,Portfolio_Cost_CV,"146,714,005","153,102,543","123,266,202","12,777,120","123,528,028","32,585,066"
2,disorderly,Production,Portfolio_Cost_CV,"295,277,703","308,135,323","248,086,479","25,715,327","248,613,433","65,580,947"
3,hothouse,Carbon Tilt,Portfolio_Cost_CV,"58,541,894","61,091,052","49,185,740","5,098,333","49,290,214","13,002,109"
4,hothouse,Equal,Portfolio_Cost_CV,"61,016,926","63,673,857","51,265,213","5,313,880","51,374,104","13,551,812"
5,hothouse,Production,Portfolio_Cost_CV,"122,803,121","128,150,481","103,176,751","10,694,754","103,395,906","27,274,477"
6,orderly,Carbon Tilt,Portfolio_Cost_CV,"105,423,954","110,014,552","88,575,119","9,181,226","88,763,259","23,414,578"
7,orderly,Equal,Portfolio_Cost_CV,"109,881,064","114,665,743","92,319,894","9,569,390","92,515,988","24,404,498"
8,orderly,Production,Portfolio_Cost_CV,"221,147,451","230,777,132","185,803,710","19,259,425","186,198,370","49,116,676"


In [248]:
# %%
def convergence_profile(costs: np.ndarray, *, alpha: float, min_samples: int = 50, step: int = 50) -> pd.DataFrame:
    metrics = []
    for n in range(min_samples, len(costs) + 1, step):
        sample = costs[:n]
        var = np.quantile(sample, alpha)
        cvar = sample[sample >= var].mean()
        metrics.append({"Samples": n, "VaR": var, "CVaR": cvar})
    return pd.DataFrame(metrics)


horizon_df = mc_results[mc_results["Year"] == horizon].copy()
conv_frames = []
for (scen_name, strategy), group in horizon_df.groupby(["Scenario", "Strategy"]):
    costs = group.sort_values("MC_id")[cost_column].values
    conv = convergence_profile(costs, alpha=alpha)
    conv["Scenario"] = scen_name
    conv["Strategy"] = strategy
    conv_frames.append(conv)

conv_df = pd.concat(conv_frames, ignore_index=True)
conv_long = conv_df.melt(
    id_vars=["Samples", "Scenario", "Strategy"],
    value_vars=["VaR", "CVaR"],
    var_name="Metric",
    value_name="Value",
)

fig = px.line(
    conv_long,
    x="Samples",
    y="Value",
    color="Strategy",
    facet_row="Metric",
    facet_col="Scenario",
    title=f"Convergence of VaR/CVaR estimates at horizon {horizon}",
    height=600,
)
fig.update_traces(mode="lines+markers")
fig.show()

In [249]:
# %%
fig = px.histogram(
    mc_results[mc_results["Year"] == horizon],
    x=cost_column,
    color="Strategy",
    facet_col="Scenario",
    nbins=40,
    marginal="box",
    title=f"Distribution of portfolio cost at {horizon} under correlated MC paths",
)
for _, row in var_df.iterrows():
    fig.add_vline(
        x=row["VaR_95"],
        line_dash="dash",
        line_color="red",
    )
fig.show()

agg_stats = (
    mc_results
    .groupby(["Scenario", "Strategy", "Year"])["Portfolio_Cost"]
    .agg(["mean", "std"])
    .reset_index()
)

fig = px.line(
    agg_stats,
    x="Year",
    y="mean",
    color="Strategy",
    facet_col="Scenario",
    error_y="std",
    title="MC expected portfolio cost with ±1 std band",
)
fig.update_traces(mode="lines+markers")
fig.show()

### 5. Clustering mines (UMAP + HDBSCAN)

# %%
output_path = export_dataframe(mc_results, "mc_results", index=False)
print(f"Saved Monte Carlo results to {output_path.as_posix()}")

In [250]:
# %%
low_mines = low.copy()
low_mines["risk_label"] = "low"

crit_mines = crit.copy()
crit_mines["risk_label"] = "critical"

mines = pd.concat([low_mines, crit_mines], ignore_index=True)

if "Emissions" not in mines.columns:
    mines["Emissions"] = np.nan
mask_low = mines["risk_label"] == "low"
mines.loc[mask_low, "Emissions"] = mines.loc[mask_low, "Cost@$100"] / 100.0  # [file:1]

features_mines = mines[["Emissions", "Intensity", "Capacity", "Cost@$100"]].fillna(0)

scaler_m = StandardScaler()
X_mines_scaled = scaler_m.fit_transform(features_mines)

reducer_m = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    random_state=DEFAULT_SEED,
)
X_mines_umap = reducer_m.fit_transform(X_mines_scaled)

clusterer_m = hdbscan.HDBSCAN(
    min_cluster_size=10,
    metric="euclidean",
    cluster_selection_method="eom",
)
mines["cluster"] = clusterer_m.fit_predict(X_mines_umap)
mines["umap1"] = X_mines_umap[:, 0]
mines["umap2"] = X_mines_umap[:, 1]

mask_cl_m = mines["cluster"] >= 0
sil_m = (
    silhouette_score(X_mines_umap[mask_cl_m], mines.loc[mask_cl_m, "cluster"])
    if mask_cl_m.sum() > 1 and mines["cluster"].nunique() > 1
    else np.nan
)
sil_m

0.9707059860229492

In [251]:
# %%
fig = px.scatter(
    mines,
    x="umap1", y="umap2",
    color="cluster",
    symbol="risk_label",
    hover_name="Mine",
    title=f"Mines – UMAP + HDBSCAN (silhouette={sil_m:.2f})"
)

fig.update_layout(
    height=600,
    margin=dict(l=40, r=40, t=60, b=120),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

fig.show()

#### 5.2 Company clustering & bootstrap stability

In [252]:
# %%
div_agg = div.groupby("Company").agg({
    "Assets at Risk": "sum",
    "Exposure@$100/t": "sum",
    "Emissions": "sum"
}).reset_index()

comp_clust = comp.merge(div_agg, on="Company", how="left", suffixes=("", "_div"))
for col in ["Assets at Risk", "Exposure@$100/t", "Emissions"]:
    comp_clust[col] = comp_clust[col].fillna(0)

features_comp = comp_clust[[
    "Total Emissions (tCO₂)",
    "Production (t)",
    "$50/t", "$100/t", "$150/t", "$200/t",
    "Portfolio Intensity",
    "Assets at Risk",
    "Exposure@$100/t",
    "Emissions"
]].fillna(0)

scaler_c = StandardScaler()
X_comp_scaled = scaler_c.fit_transform(features_comp)

def cluster_companies_umap_hdbscan(X_scaled, n_neighbors=10, min_dist=0.05,
                                   min_cluster_size=4, random_state=42):
    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=2,
        random_state=random_state,
        init="random"
    )
    X_umap = reducer.fit_transform(X_scaled)
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    labels = clusterer.fit_predict(X_umap)
    return labels, X_umap, clusterer

labels_base, X_comp_umap, clusterer_base = cluster_companies_umap_hdbscan(X_comp_scaled)
comp_clust["cluster"] = labels_base
comp_clust["umap1"] = X_comp_umap[:, 0]
comp_clust["umap2"] = X_comp_umap[:, 1]

In [253]:
# %%
n_boot = 50
n = comp_clust.shape[0]
stability_counts = np.zeros(n, dtype=int)
base_labels = labels_base.copy()
companies = comp_clust["Company"].values

for b in range(n_boot):
    idx_boot = np.random.choice(np.arange(n), size=n, replace=True)
    X_boot = X_comp_scaled[idx_boot]
    labels_boot, _, _ = cluster_companies_umap_hdbscan(
        X_boot,
        random_state=42 + b
    )
    orig_idx = idx_boot
    
    unique_boot = np.unique(labels_boot[labels_boot >= 0])
    for c_boot in unique_boot:
        mask_b = labels_boot == c_boot
        orig_in_cluster = orig_idx[mask_b]
        base_in_cluster = base_labels[orig_in_cluster]
        vals, counts = np.unique(base_in_cluster[base_in_cluster >= 0], return_counts=True)
        if len(vals) == 0:
            continue
        best_base_cluster = vals[np.argmax(counts)]
        mask_valid = base_labels[orig_in_cluster] == best_base_cluster
        stability_counts[orig_in_cluster[mask_valid]] += 1

comp_clust["cluster_stability"] = stability_counts / n_boot

In [254]:
# %%
mask_cl_c = comp_clust["cluster"] >= 0

if mask_cl_c.sum() > 1 and comp_clust["cluster"].nunique() > 1:
    sil_c = silhouette_score(
        X_comp_umap[mask_cl_c],
        comp_clust.loc[mask_cl_c, "cluster"],
    )
else:
    sil_c = np.nan

fig = px.scatter(
    comp_clust,
    x="umap1", y="umap2",
    color="cluster",
    size="cluster_stability",
    hover_name="Company",
    title=f"Companies – UMAP + HDBSCAN (silhouette={sil_c:.2f})"
)
fig.update_layout(height=600)
fig.show()

### 6. Refined risk label & supervised model

#### 6.1 Refined risk label from cost & intensity

In [255]:
# Build cost-by-horizon features
horizons = [2030, 2040]

cost_horizons = (
    paths_df[paths_df["Year"] <= max(horizons)]
    .groupby(["Scenario", "Company", "Year"])["Cost_t"]
    .sum()
    .reset_index()
)

rows = []
for scen in cost_horizons["Scenario"].unique():
    for comp_name in cost_horizons["Company"].unique():
        sub = cost_horizons[(cost_horizons["Scenario"] == scen) &
                            (cost_horizons["Company"] == comp_name)]
        if sub.empty:
            continue
        for H in horizons:
            val = sub[sub["Year"] <= H]["Cost_t"].sum()
            rows.append({
                "Company": comp_name,
                "Scenario": scen,
                f"Cost_cum_{H}": val
            })

cost_features = pd.DataFrame(rows)
cost_pivot = cost_features.pivot_table(
    index="Company",
    columns="Scenario",
    values=[f"Cost_cum_{H}" for H in horizons],
    aggfunc="sum"
)
cost_pivot.columns = [f"{c[0]}_{c[1]}" for c in cost_pivot.columns]
cost_pivot = cost_pivot.reset_index()

label_df = comp[["Company", "Portfolio Intensity"]].merge(
    cost_pivot, on="Company", how="left"
)

cost_cols = [c for c in label_df.columns if c.startswith("Cost_cum_")]
scaler_cost = MinMaxScaler()
label_df[cost_cols] = scaler_cost.fit_transform(label_df[cost_cols].fillna(0))

label_df["score_composite"] = label_df[cost_cols].mean(axis=1) * label_df["Portfolio Intensity"]
thr = label_df["score_composite"].quantile(0.8)
label_df["high_risk_refined"] = (label_df["score_composite"] >= thr).astype(int)

label_df[["Company", "score_composite", "high_risk_refined"]].head()

,Company,score_composite,high_risk_refined
0,FreePort-McMoran Inc,0.010375,1
1,Government of Iran,0.022365,1
2,Qatar Investment Authority,0.017315,1
3,Kazakhmys Holding LLP,0.002586,0
4,The Vanguard Group Inc,0.001934,0


#### 6.2 RandomForest classifier & UMAP visualization

In [256]:
# %%
cost_total = paths_df.groupby("Company")["Cost_t"].sum().reset_index()
cost_total.rename(columns={"Cost_t": "Cost_total_all_scen"}, inplace=True)

comp_ml = comp_clust.merge(cost_total, on="Company", how="left")
comp_ml = comp_ml.merge(label_df[["Company", "score_composite", "high_risk_refined"]],
                        on="Company", how="left")

feature_cols = [
    "Total Emissions (tCO₂)",
    "Production (t)",
    "$50/t", "$100/t", "$150/t", "$200/t",
    "Portfolio Intensity",
    "Assets at Risk",
    "Exposure@$100/t",
    "Emissions",
    "cluster_stability",
    "score_composite"
]

X = comp_ml[feature_cols].fillna(0)
y = comp_ml["high_risk_refined"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    class_weight="balanced"
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

importances = pd.Series(clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
importances

              precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      0.50      0.67         2

    accuracy                           0.89         9
   macro avg       0.94      0.75      0.80         9
weighted avg       0.90      0.89      0.87         9

ROC-AUC: 1.0


score_composite           0.263300
Portfolio Intensity       0.093648
$50/t                     0.091371
$150/t                    0.082891
Total Emissions (tCO₂)    0.082043
Exposure@$100/t           0.068541
$100/t                    0.066809
$200/t                    0.064413
Emissions                 0.061157
Production (t)            0.059137
cluster_stability         0.034034
Assets at Risk            0.032657
dtype: float64

In [257]:
# %%
fig = px.bar(
    x=importances.values,
    y=importances.index,
    orientation="h",
    title="Feature importance – refined high_risk model"
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [258]:
# %%
y_proba_all = clf.predict_proba(X)[:, 1]
y_pred_all = clf.predict(X)

comp_ml["pred_high_risk"] = y_pred_all
comp_ml["pred_score"] = y_proba_all

viz_df = comp_clust[["Company", "umap1", "umap2", "cluster"]].merge(
    comp_ml[["Company", "high_risk_refined", "pred_high_risk", "pred_score"]],
    on="Company", how="left"
)

fig = px.scatter(
    viz_df,
    x="umap1", y="umap2",
    color="pred_high_risk",
    hover_name="Company",
    title="UMAP – predicted refined high_risk (0/1)",
    color_discrete_map={0: "blue", 1: "red"}
)
fig.show()

fig = px.scatter(
    viz_df,
    x="umap1", y="umap2",
    color="pred_score",
    hover_name="Company",
    title="UMAP – continuous risk score",
    color_continuous_scale="Magma"
)
fig.show()